In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import pathlib as Path
import os

In [2]:
images_path = 'images'
anno_path = "annotations"

In [6]:
def filelist(root, file_type):
    """Returns a fully-qualified list of filenames under root directory"""
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

def generate_train_df (anno_path):
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = images_path + '/'+ root.find("./filename").text
        anno['width'] = root.find("./size/width").text
        anno['height'] = root.find("./size/height").text
        anno['class'] = root.find("./object/name").text
        anno['xmin'] = int(root.find("./object/bndbox/xmin").text)
        anno['ymin'] = int(root.find("./object/bndbox/ymin").text)
        anno['xmax'] = int(root.find("./object/bndbox/xmax").text)
        anno['ymax'] = int(root.find("./object/bndbox/ymax").text)
        anno_list.append(anno)
    return pd.DataFrame(anno_list)

In [7]:
df_train = generate_train_df(anno_path)
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,images/road0.png,267,400,trafficlight,98,62,208,232
1,images/road1.png,400,283,trafficlight,154,63,258,281
2,images/road10.png,400,267,trafficlight,106,3,244,263
3,images/road100.png,400,385,speedlimit,35,5,363,326
4,images/road101.png,400,200,speedlimit,195,7,392,194


In [8]:
#label encode target
class_dict = {'speedlimit': 0, 'stop': 1, 'crosswalk': 2, 'trafficlight': 3}
df_train['class'] = df_train['class'].apply(lambda x:  class_dict[x])

In [9]:
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,images/road0.png,267,400,3,98,62,208,232
1,images/road1.png,400,283,3,154,63,258,281
2,images/road10.png,400,267,3,106,3,244,263
3,images/road100.png,400,385,0,35,5,363,326
4,images/road101.png,400,200,0,195,7,392,194


In [11]:
# Data format for faster R-Cnn   :    ['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
train_df= df_train.loc[:,['filename','xmin','xmax','ymin','ymax','class']]
train_df

,filename,xmin,xmax,ymin,ymax,class
0,images/road0.png,98,208,62,232,3
1,images/road1.png,154,258,63,281,3
2,images/road10.png,106,244,3,263,3
3,images/road100.png,35,363,5,326,0
4,images/road101.png,195,392,7,194,0
...,...,...,...,...,...,...
872,images/road95.png,90,268,101,257,1
873,images/road96.png,209,326,58,179,1
874,images/road97.png,42,130,66,154,1
875,images/road98.png,138,376,17,245,1


In [12]:
train_df.to_csv('train.csv')

In [13]:
train_df['class'].value_counts()

0    652
2     88
1     76
3     61
Name: class, dtype: int64

In [14]:
import cv2

In [18]:
# Writing train.csv to annotation.txt

train_df = pd.read_csv('train.csv')

['filename','xmin','xmax','ymin','ymax','class']

# for training
with open("annotation.txt", "w+") as f:
  for idx, row in train_df.iterrows():
      img = cv2.imread(row['filename'])
      #print(img)  #none 
      height, width = img.shape[:2]
      x1 = int(row['xmin'] * width)
      x2 = int(row['xmax'] * width)
      y1 = int(row['ymin'] * height)
      y2 = int(row['ymax'] * height)
      
      fileName = os.path.join(row['filename'])
      className = row['class']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + str(className) + '\n')